### Journée X/Stra-Dev Intégration continue

21 novembre 2017  

--- 

# L'intégration continue avec GitLab

![image.png](tanuki.png)

## *Session pratique*

---

Matthieu Boileau - <matthieu.boileau@math.unistra.fr>

*Contenu sous licence [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0)*

# GitLab CI

GitLab CI propose une chaîne complète d'intégration continue intégrée à son système de forge logicielle.

![](cicd_pipeline_infograph.png)
<span style="font-size:0.7em;"><https://docs.gitlab.com/ee/ci/README.html></span>

**Les avantages :**

- la simplicité de mise en oeuvre
- la robustesse d'une solution intégrée

## Le gitlab runner

### Introduction

- `gitlab-runner` est un démon installé sur une machine destinée à exécuter les tâches d'intégration continue.
- Pour chaque *runner* enregistré, `gitlab-runner` écoute l'instance de gitlab correspondante dans l'attente d'une tâche
- Cette tâche est déclenchée par des évènements (`push`, `merge`, etc.) sur le projet avec lequel il est lié
- Ce lien est créé à l'enregistrement du *runner* par un token associé au projet.

[**Doc en ligne**](https://docs.gitlab.com/ce/ci/runners/#shared-vs-specific-runners)

### Runners : *shared* vs *specific*

- *shared* : n'importe quel projet peut s'en servir. Seul l'administrateur du GitLab peut enregistrer des runners partagés.
- *specific* : seul le créateur du runner peut s'en servir dans d'autres projets

> **Attention** : les runners partagés sont pratiques mais peuvent présenter des [problèmes de sécurité](https://docs.gitlab.com/ee/ci/runners/README.html#be-careful-with-sensitive-information).


### Runners : *locked* vs *unlocked runners*

- *locked* : le runner ne peut être utilisé que pour le projet avec lequel il est lié
- *unlocked* : le créateur du runner peut l'utiliser dans ses autres projets

### Installer `gitlab-runner`

La **[procédure](https://docs.gitlab.com/runner/install/)** dépend du système d'exploitation.

## Fork du TP

Faire un *fork* du projet sur votre compte GitLab en allant sur la page d'accueil du projet

![fork.png](fork.png)

ou en suivant [ce lien](https://git.unistra.fr/m.boileau/tp-gitlab-ci/forks/new).

## Clone du TP

- Pour paramétrer les commandes de ce notebook, positionner les variables d'environnement suivantes :

In [ ]:
export USERNAME="m.boileau"  # votre login sur git.unistra.fr
export TPBASEDIR=$HOME       # le répertoire où vous souhaitez installer le TP
export TPDIR=$TPBASEDIR/tp-gitlab-ci

- Cloner localement votre projet

In [ ]:
cd $TPBASEDIR
git clone git@git.unistra.fr:$USERNAME/tp-gitlab-ci.git

## Enregistrer  un nouveau runner

> Si `gitlab-runner` n'est pas installé sur votre machine, rdv à la section **Exo1**

Engistrer un runner consiste à créer un nouveau runner associé à un projet.
Si vous n'êtes pas administrateur du GitLab, il s'agit forcément d'un runner spécifique. Celui-ci se trouve dans 
votre projet, rubrique :

```
Settings > CI / CD > Runners settings
```

Dans le terminal, maintenant :

In [ ]:
export CI_SERVER_URL=https://git.unistra.fr/  # Do not change for this session
export RUNNER_NAME=a_runner_for_$USERNAME
export REGISTRATION_TOKEN=xxx  # Replace by the value provided by the runner setting page
export REGISTER_NON_INTERACTIVE=true
#gitlab-runner register --executor shell        # sous MacOS
#sudo gitlab-runner register --executor shell   # sous Linux

Ou bien lancez la commande en mode interactif et en suivant pas à pas la [procédure](http://docs.gitlab.com/runner/register/#registering-runners):
```
gitlab-runner register       # sous MacOS
sudo gitlab-runner register  # sous Linux
```

- Retournez sur la page de paramètrage des runners
- Repérez votre nouveau runner
- Editez-le et ajouter les tags `specific, shell`

Votre runner est maintenant prêt à travailler. Nous allons le tester en lui soumettant une tâche GitLab CI déclenchée par un `git push` sur votre projet.

## Exo1 : helloworld en bash

### Mettre en place une action d'intégration continue

Allez dans le répertoire racine du clone local de votre projet et basculez sur la branche `exo1`

In [ ]:
cd $TPDIR

In [ ]:
git pull
git stash
git checkout exo1
git checkout exo1-start .gitlab-ci.yml  # Clean previous modifications

Repérer le fichier `.gitlab-ci.yml`

In [ ]:
ls -al

L'éditer avec l'éditeur de votre choix

In [ ]:
cat .gitlab-ci.yml

- `helloworld` indique le nom du job d'intégration continue
- `tags` permet de sélectionner des runners sur la base de mot-clés
- `scripts` correspond aux lignes de commande `bash` que vous souhaitez exécuter

### Si vous souhaitez utiliser un runner partagé

Dans la rubrique :

```
Settings > CI / CD > Runners settings
```

Notez la présence de plusieurs runners partagés dont `clu1-tp-gitlab-ci` qui porte les tags `shell` et `docker` mais pas `specific`. Pour cibler ce runner, retirer le tag `specific` du fichier `.gitlab-ci.yml`

### Déclenchez une action d'intégration continue

Modifiez la ligne de script, par exemple:

In [ ]:
echo '    - echo "bonjour, monde"' >> .gitlab-ci.yml
cat .gitlab-ci.yml

Enregistrez avec git

In [ ]:
git add .gitlab-ci.yml
git commit -m "Update .gitlab-ci.yml"

Poussez vos modifications

In [ ]:
git push

Sur GitLab, observez l'exécution de votre job dans la rubrique de votre projet :
```
CI / CD > Pipelines
```

### Provoquez une erreur

Dans `.gitlab-ci.yml`, introduisez une erreur dans une des lignes du script et observer l'effet dans `CI / CD > Pipelines` après avoir poussé les modifications.

> **Note :** les fichiers peuvent s'éditer directement dans GitLab.

## Exo2 : helloworld en C

### Description

On écrit le programme `helloworld` en C qui doit afficher `hello, world`.

### Mettre en place une action de *build*

Dans votre clone local, basculez sur la branche `exo2`

In [ ]:
cd $TPDIR
git stash  # si vous avez des modifications non enregistrées dans exo1
git checkout exo2
git rm .gitlab-ci.yml  # Clean previous commits

Listez le contenu du répertoire

In [ ]:
ls -al

Vérifiez que le progamme compile

In [ ]:
make

On crée le fichier `.gitlab-ci.yml` avec une étape (`stage`) de `build`. Il doit contenir :
```yaml
build_hello:
  stage: build
  tags:
    - shell
  script:
    - make
```

In [ ]:
cat > .gitlab-ci.yml <<- EOM
build_hello:
  stage: build
  tags:
    - shell
  script:
    - make
EOM

On vérifie le contenu

In [ ]:
cat .gitlab-ci.yml

On enregistre et on pousse.

In [ ]:
git add .gitlab-ci.yml
git commit -m "Add .gitlab-ci.yml with a build stage"

In [ ]:
git push

Sur GitLab, observez l'exécution de votre job dans la rubrique de votre projet :
```
CI / CD > Pipelines
```

### Ajouter une action de *test*

On souhaite ajouter une étape de test réalisée avec `make test`. Tester l'exécution du test (il ne doit pas renvoyer d'erreur) :

In [ ]:
make test

Il faut ajouter le contenu suivant :
```yaml
test_hello:
  stage: test
  tags:
    - shell
  script:
    - make test
```

In [ ]:
cat >> .gitlab-ci.yml <<- EOM

test_hello:
  stage: test
  tags:
    - shell
  script:
    - make test
EOM

In [ ]:
cat .gitlab-ci.yml

On enregistre, on pousse et on vérifie l'exécution de la chaîne d'intégration continue dans `CI / CD > Pipelines`


In [ ]:
git add .gitlab-ci.yml
git commit -m "Add a test stage to .gitlab-ci.yml"

In [ ]:
git push

### Provoquez des erreurs

Modifier les fichiers


- de build : notez que le test n'est pas exécuté
- de test : le


## Récupérer un artifact